In [ ]:
from sklearn.utils import resample
# from imblearn.over_sampling import SMOTE, SVMSMOTE
from sklearn.preprocessing import FunctionTransformer

import warnings
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


# Suppress all warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer, Normalizer, MaxAbsScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.decomposition import PCA, NMF, SparsePCA, KernelPCA
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, SelectPercentile, VarianceThreshold

from sklearn.svm import SVC
from fairlearn.metrics import demographic_parity_difference
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier
# !pip install scikit-learn
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [ ]:
classifier_S = SVC(kernel='rbf', C=1.0, gamma='scale', verbose=False)
classifier_R = RandomForestClassifier(verbose=False)
classifier_D = DecisionTreeClassifier()
classifier_G = GradientBoostingClassifier(verbose=False)
classifier_K = KNeighborsClassifier()
classifier_GNB = GaussianNB()
classifier_GB = GradientBoostingClassifier(verbose=False)
classifier_NN = MLPClassifier()
classifier_IDA = QuadraticDiscriminantAnalysis()
classifier_QDA = QuadraticDiscriminantAnalysis()
classifier_ADB = AdaBoostClassifier()
classifier_GP = GaussianProcessClassifier()
classifier_XGBC = XGBClassifier(verbosity=0)
classifier_LGBM = LGBMClassifier()
classifier_BC = BaggingClassifier(verbose=False)
classifier_CB = CatBoostClassifier(iterations=100, verbose=False)

classifiers = [
     ('SVC', classifier_S),
     ('RF', classifier_R),
     ('DT', classifier_D),
       ('GB', classifier_G),
     ('KNN', classifier_K),
       ('GNB', classifier_GNB),
     ('GBC', classifier_GB),
     ('NN', classifier_NN),
     ('IDA', classifier_IDA),
     ('QDA', classifier_QDA),  
      ('ADB', classifier_ADB),
#      ('GP', classifier_GP),
     ('XGB', classifier_XGBC),
     ('LGB', classifier_LGBM),
     ('BC', classifier_BC),
     ('CB', classifier_CB),     
]


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

In [ ]:

def Fairness_Metrics_Computation(y1, y2, y, attribute):
    # Convert inputs to pandas Series for easier handling
    y1 = pd.Series(y1)
    y2 = pd.Series(y2)
    y = pd.Series(y)
    attribute = pd.Series(attribute)

    # Identify privileged and unprivileged groups
    privileged = (attribute == 'male') | (attribute == 1)
    unprivileged = ~privileged

    # Counts of privileged and unprivileged groups
    count_privileged = np.sum(privileged)
    count_unprivileged = np.sum(unprivileged)

    # SPD Calculation
    SFC_SPD_P = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                        for y1_i, y2_i, attr in zip(y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_SPD_NP = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                         for y1_i, y2_i, attr in zip(y1, y2, attribute) if attr != 'male' and attr != 1])

    SFR_SPD_p = SFC_SPD_P / count_privileged if count_privileged > 0 else 0
    SFR_SPD_np = SFC_SPD_NP / count_unprivileged if count_unprivileged > 0 else 0
    SPD = SFR_SPD_np - SFR_SPD_p

    # EOD Calculation
    SFC_EOD_P = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_EOD_NP = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                         for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])

    P_Y1 = np.sum((y == 1) & privileged)
    NP_Y1 = np.sum((y == 1) & unprivileged)

    SFR_EOD_p = SFC_EOD_P / P_Y1 if P_Y1 > 0 else 0
    SFR_EOD_np = SFC_EOD_NP / NP_Y1 if NP_Y1 > 0 else 0
    EOD = SFR_EOD_np - SFR_EOD_p

    # AOD Calculation
    SFC_TP_P = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_TP_NP = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])

    P_Y0 = np.sum((y == 0) & privileged)
    NP_Y0 = np.sum((y == 0) & unprivileged)

    SFR_AOD_p = (SFC_TP_P / P_Y1 if P_Y1 > 0 else 0) + (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0)
    SFR_AOD_np = (SFC_TP_NP / NP_Y1 if NP_Y1 > 0 else 0) + (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0)
    AOD = (SFR_AOD_np - SFR_AOD_p) / 2

    # ERD Calculation
    SFC_FN_P = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_FN_NP = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])

    SFR_ERR_p = (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0) + (SFC_FN_P / P_Y1 if P_Y1 > 0 else 0)
    SFR_ERR_np = (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0) + (SFC_FN_NP / NP_Y1 if NP_Y1 > 0 else 0)
    ERD = SFR_ERR_np - SFR_ERR_p

    return SPD, EOD, AOD, ERD

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

# Load the dataset
file_path = 'C:/Users/Saadia/FairPreprocessing/data/bank/bank-additional-full.csv'
na_values = ['unknown', 'nan', '']  # More clearly defined missing value indicators
df = pd.read_csv(file_path, sep=';', na_values=na_values)

# Apply custom transformations
df['age'] = df['age'].apply(lambda x: float(x >= 25))  # Transforming 'age' into a binary feature
X = df.drop(columns=['y'])
y = df['y']

# Convert 'y' labels to numeric
y = y.map({'no': 0, 'yes': 1})

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

# Define a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ],
    remainder='passthrough'
)

# Manually compute sample weights for handling class imbalance
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# Define a pipeline incorporating Random Forest with class weight balancing
pipeline_1 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        random_state=42,
        n_estimators=200,  # Increased estimators
        max_depth=15,      # Increased depth for more complex trees
        min_samples_split=10,  # Require more samples to split
        class_weight='balanced'  # Handle class imbalance
    ))
])

# Define a second pipeline without class weight balancing and different parameters
pipeline_2 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        random_state=42,
        n_estimators=80,   # Reduced estimators
        max_depth=10,      # Shallower depth
        min_samples_leaf=4 # Increased leaf samples to prevent overfitting
    ))
])

# Fit the pipelines using the training data
pipeline_1.fit(X_train, y_train, classifier__sample_weight=sample_weights)
pipeline_2.fit(X_train, y_train)

# Predict and evaluate on the test data
y_pred_1 = pipeline_1.predict(X_test)
y_pred_2 = pipeline_2.predict(X_test)

accuracy_class_weight = accuracy_score(y_test, y_pred_1)
accuracy_no_class_weight = accuracy_score(y_test, y_pred_2)

print(f'Accuracy with Class Weighting: {accuracy_class_weight:.5f}')
print(f'Accuracy without Class Weighting: {accuracy_no_class_weight:.5f}')


In [ ]:
A1 = accuracy_score(y_test, y_pred_1)
A2 = accuracy_score(y_test,y_pred_2)


f1 = f1_score(y_test, y_pred_1)
f2 = f1_score(y_test, y_pred_2)
A1,A2

In [ ]:
from sklearn.dummy import DummyClassifier

pipeline_baseline = DummyClassifier(strategy='most_frequent')
pipeline_baseline.fit(X_train, y_train)
y_pred_baseline = pipeline_baseline.predict(X_test)
mispred_baseline = (y_test != y_pred_baseline).sum()


In [ ]:
from sklearn.dummy import DummyClassifier
import matplotlib.pyplot as plt

def calculate_biases(mispred_p1, mispred_p2, mispred_baseline):
    """
    Calculate individual biases and total bias.
    """
    # Calculating individual biases
    BD = mispred_baseline
    BStage = mispred_p1 - mispred_p2  # Preprocessing stage bias
    BC = mispred_p2 - mispred_baseline  # Classifier bias

    # Calculating Stage-Classifier Interaction term
    Stage_Classifier_Interaction = mispred_p1 - (BD + BStage + BC)

    # Total bias calculation
    TB = BD + BStage + BC + Stage_Classifier_Interaction

    return {
        'BD': BD,
        'BC': BC,
        'BStage': BStage,
        'Stage_Classifier_Interaction': Stage_Classifier_Interaction,
        'TB': TB,
        'TB matches mispred_p1': TB == mispred_p1
    }

def plot_biases(biases, mispred_p1):
    """
    Plot the biases and observed mispredictions.
    """
    labels = [
        'Data (BD)', 
         
        'Preprocessing Stage (BStage)', 
        'Stage-Classifier Interaction', 
        'Classifier (BC)',
        'Total Bias (TB)', 
        'Observed Bias (mispred_p1)'
    ]
    
    values = [
        biases['BD'], 
         
        biases['BStage'], 
        biases['Stage_Classifier_Interaction'], 
        biases['BC'],
        biases['TB'], 
        mispred_p1
    ]

    colors = ['blue', 'orange', 'green', 'red', 'pink', 'gray']

    plt.figure(figsize=(12, 8))
    bars = plt.barh(labels, values, color=colors)
    plt.xlabel('Bias / Mispredictions')
    plt.title('Bias Analysis and Observed Mispredictions')

    # Adding annotations
    for bar in bars:
        width = bar.get_width()
        label_x_pos = width if width < 0 else width
        plt.text(
            label_x_pos, bar.get_y() + bar.get_height() / 2,
            f'{width:.2f}', ha='center', va='center',
            bbox=dict(facecolor='white', alpha=0.7)
        )

    plt.axvline(0, color='black', linewidth=0.8)
    plt.xlim(min(values) - 10, max(values) + 10)  # Adding some padding to the x-axis
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    plt.show()

# Baseline model for comparison


# Calculate mispredictions
aligned_indices = X_test.notna().all(axis=1)

mispred_p1 = sum(y_test != y_pred_1)
mispred_p2 = sum(y_test != y_pred_2)
# mispred_baseline = sum(y_test != y_pred_baseline)

# Calculate biases
biases = calculate_biases(mispred_p1, mispred_p2, mispred_baseline)

# Plot the biases
plot_biases(biases, mispred_p1)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming y_pred_1, y_pred_2, y_pred_baseline, and y_test are already defined
# Convert y_test and predictions to numpy arrays to avoid indexing issues
y_test = np.array(y_test)
y_pred_1 = np.array(y_pred_1)
y_pred_2 = np.array(y_pred_2)
y_pred_baseline = np.array(y_pred_baseline)

# Initialize volatility arrays
volatility_LE = np.zeros(len(y_test))
volatility_classifier = np.zeros(len(y_test))

# Calculate Volatility Scores for LE (Vol_St1)
for i in range(len(y_test)):
    if y_pred_1[i] == y_test[i] and y_pred_2[i] == y_test[i]:
        volatility_LE[i] = 0
    elif y_pred_1[i] == y_test[i] and y_pred_2[i] != y_test[i]:
        volatility_LE[i] = 1
    elif y_pred_1[i] != y_test[i] and y_pred_2[i] == y_test[i]:
        volatility_LE[i] = 2
    elif y_pred_1[i] != y_test[i] and y_pred_2[i] != y_test[i] and y_pred_1[i] == y_pred_2[i]:
        volatility_LE[i] = 3
    elif y_pred_1[i] != y_test[i] and y_pred_2[i] != y_test[i] and y_pred_1[i] != y_pred_2[i]:
        volatility_LE[i] = 4

# Volatility score calculation for Classifier (Vol_Cl)
for i in range(len(y_test)):
    if y_pred_1[i] == y_test[i] and y_pred_baseline[i] == y_test[i]:
        volatility_classifier[i] = 0
    elif y_pred_1[i] == y_test[i] and y_pred_baseline[i] != y_test[i]:
        volatility_classifier[i] = 1
    elif y_pred_1[i] != y_test[i] and y_pred_baseline[i] == y_test[i]:
        volatility_classifier[i] = 2
    elif y_pred_1[i] != y_test[i] and y_pred_baseline[i] != y_test[i] and y_pred_1[i] == y_pred_baseline[i]:
        volatility_classifier[i] = 3
    elif y_pred_1[i] != y_test[i] and y_pred_baseline[i] != y_test[i] and y_pred_1[i] != y_pred_baseline[i]:
        volatility_classifier[i] = 4

# Summarize volatility scores
volatility_summary = {
    'Condition': ['Vol_St1=0', 'Vol_St1=1', 'Vol_St1=2', 'Vol_St1=3', 'Vol_St1=4',
                  'Vol_Cl=0', 'Vol_Cl=1', 'Vol_Cl=2', 'Vol_Cl=3', 'Vol_Cl=4'],
    'Count': [np.sum(volatility_LE == 0), np.sum(volatility_LE == 1), np.sum(volatility_LE == 2), np.sum(volatility_LE == 3), np.sum(volatility_LE == 4),
              np.sum(volatility_classifier == 0), np.sum(volatility_classifier == 1), np.sum(volatility_classifier == 2), np.sum(volatility_classifier == 3), np.sum(volatility_classifier == 4)]
}

df_volatility = pd.DataFrame(volatility_summary)

# Plot the volatility scores
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Condition', y='Count', data=df_volatility, palette='viridis')

plt.title('Volatility Score Analysis')
plt.xlabel('Condition')
plt.ylabel('Count')

# Add labels to each bar
for bar in ax.patches:
    ax.text(bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            f'{int(bar.get_height())}',
            ha='center', va='bottom', fontsize=11)

plt.show()


In [ ]:
from typing import Callable, Dict, List, Tuple, Any, Optional
import numpy as np
from sklearn.metrics import accuracy_score

# Optional: SMOTE if you plan to oversample (safe fallback if not installed)
try:
    from imblearn.over_sampling import SMOTE  # noqa
except Exception:
    SMOTE = None  # oversampling step will be skipped if None


# ---------- Mitigation framework (no hard-coding of stages) ----------
MitigationFn = Callable[
    [Any, np.ndarray, np.ndarray, Dict[str, Any]],
    Tuple[Any, np.ndarray, np.ndarray, Dict[str, Any]]
]
# Signature: fn(pipeline, X_train, y_train, context) -> (pipeline, X_train, y_train, fit_params_delta)

def run_mitigations_and_predict(
    pipeline: Any,
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test:  np.ndarray,
    y_test:  Optional[np.ndarray] = None,
    mitigations: Optional[List[Dict[str, Any]]] = None,
    base_fit_params: Optional[Dict[str, Any]] = None,
    context: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Fits pipeline, applies mitigation callables, then refits and re-predicts.
    Returns pre/post metrics & predictions. No random post-hoc tweaks.
    """
    base_fit_params = dict(base_fit_params or {})
    context = dict(context or {})
    mitigations = mitigations or []

    # 1) Baseline fit & predict (pre-mitigation)
    pipeline.fit(X_train, y_train, **base_fit_params)
    y_pred_pre = pipeline.predict(X_test)

    metrics_pre = {}
    if y_test is not None:
        metrics_pre["accuracy"] = float(accuracy_score(y_test, y_pred_pre))

    # 2) Apply mitigations (may update pipeline/data/fit_params)
    applied = []
    X_train_m, y_train_m = X_train, y_train
    fit_params = dict(base_fit_params)

    for step in mitigations:
        name = step.get("name", "<unnamed>")
        fn: MitigationFn = step["fn"]
        params = step.get("params", {})
        ctx = dict(context); ctx.update(params)

        pipeline, X_train_m, y_train_m, delta = fn(pipeline, X_train_m, y_train_m, ctx)
        if delta:
            fit_params.update(delta)
        applied.append({"name": name, "fit_params_delta_keys": list((delta or {}).keys())})

    # 3) Refit post-mitigation & predict
    pipeline.fit(X_train_m, y_train_m, **fit_params)
    y_pred_post = pipeline.predict(X_test)

    metrics_post = {}
    if y_test is not None:
        metrics_post["accuracy"] = float(accuracy_score(y_test, y_pred_post))

    return {
        "y_pred_pre": y_pred_pre,
        "y_pred_post": y_pred_post,
        "metrics_pre": metrics_pre,
        "metrics_post": metrics_post,
        "applied_mitigations": applied,
        "pipeline_final": pipeline
    }


# ---------- Example mitigation primitives (compose as you like) ----------
def mitigation_reweighing(pipeline, X_train, y_train, context):
    """
    Adds per-sample weights into fit params (no data mutation).
    Provide either:
      - context["sample_weights"] (np.ndarray aligned with y_train), or
      - context["weight_fn"](X_train, y_train, context) -> np.ndarray
    You can change which estimator receives weights via context["weight_param"]
      e.g., "classifier__sample_weight" or "clf__sample_weight".
    """
    if "sample_weights" in context:
        w = np.asarray(context["sample_weights"])
    elif callable(context.get("weight_fn", None)):
        w = np.asarray(context["weight_fn"](X_train, y_train, context))
    else:
        raise ValueError("mitigation_reweighing needs 'sample_weights' or 'weight_fn' in context.")

    target_param = context.get("weight_param", "classifier__sample_weight")
    return pipeline, X_train, y_train, {target_param: w}


def mitigation_oversample(pipeline, X_train, y_train, context):
    """
    Uses a provided oversampler (e.g., SMOTE) from context["oversampler"].
    """
    oversampler = context.get("oversampler", None)
    if oversampler is None:
        # If SMOTE not available or not provided, leave data unchanged
        return pipeline, X_train, y_train, {}
    X_res, y_res = oversampler.fit_resample(X_train, y_train)
    return pipeline, X_res, y_res, {}


def mitigation_encoder_tweak(pipeline, X_train, y_train, context):
    """
    Placeholder to swap/modify an encoder inside your pipeline if needed.
    Example:
      pipeline.set_params(preprocess__encoder=YourEncoder(**context.get("encoder_params", {})))
    """
    return pipeline, X_train, y_train, {}


# ---------- A simple, sensible reweighting function (group × label inverse prevalence) ----------
def inverse_prevalence_weights(X, y, ctx):
    """
    Compute weights ~ 1 / P(y, s) using ctx["s_train"] (sensitive attribute, aligned with y).
    """
    s = np.asarray(ctx.get("s_train", None))
    if s is None:
        raise ValueError("inverse_prevalence_weights requires 's_train' in context.")
    y = np.asarray(y)
    pairs, counts = np.unique(np.column_stack([y, s]), axis=0, return_counts=True)
    freq = {tuple(k): v for k, v in zip(map(tuple, pairs), counts)}
    total = len(y)
    w = np.empty_like(y, dtype=float)
    for i, (yy, ss) in enumerate(zip(y, s)):
        p = freq[(yy, ss)] / total
        w[i] = 1.0 / max(p, 1e-12)
    # Normalize to mean 1.0 (optional)
    w /= w.mean()
    return w


# =========================
# PRODUCE THE NEW PREDICTIONS
# =========================
# Assumptions: you already have:
#   pipeline_1, pipeline_2, pipeline_3  (sklearn Pipelines/estimators)
#   X_train, y_train, X_test, y_test     (arrays/Series)
#   s_train                               (sensitive attribute aligned with y_train)
#
# If you don't want oversampling, just remove the mitigation_oversample step below.

mitigations_cfg = [
    {"name": "Reweighing", "fn": mitigation_reweighing,
     "params": {
         "weight_param": "classifier__sample_weight",   # adjust to your pipeline
         "weight_fn": inverse_prevalence_weights
     }},
    {"name": "Oversample", "fn": mitigation_oversample,
     "params": {"oversampler": SMOTE(random_state=42) if SMOTE is not None else None}},
    {"name": "Encoder tweak", "fn": mitigation_encoder_tweak, "params": {}},
]

common_context = {"s_train": s_train, "seed": 42}

res_1 = run_mitigations_and_predict(
    pipeline=pipeline_1,
    X_train=X_train, y_train=y_train,
    X_test=X_test,  y_test=y_test,
    mitigations=mitigations_cfg,
    base_fit_params={},
    context=common_context
)
y_pred_1n = res_1["y_pred_post"]

res_2 = run_mitigations_and_predict(
    pipeline=pipeline_2,
    X_train=X_train, y_train=y_train,
    X_test=X_test,  y_test=y_test,
    mitigations=mitigations_cfg,
    base_fit_params={},
    context=common_context
)
y_pred_2n = res_2["y_pred_post"]


In [ ]:
from __future__ import annotations
from typing import Dict, List, Tuple, Any, Optional, Sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.metrics import accuracy_score


# =========================
# Helpers for stage outputs
# =========================
def _truncate_pipeline(pipeline, stage_name: str):
    """
    Return a shallow-cloned pipeline that runs up to and including `stage_name`.
    Special names:
      - "__raw__": no transform, just return raw X
      - "__end__": full pipeline
    """
    if stage_name == "__raw__":
        return None
    if stage_name == "__end__":
        return clone(pipeline)
    steps = []
    for name, est in pipeline.steps:
        steps.append((name, est))
        if name == stage_name:
            break
    from sklearn.pipeline import Pipeline
    return Pipeline(steps=steps)

def _fit_transform_at_stage(pipeline, X: pd.DataFrame, stage_name: str) -> pd.DataFrame:
    if stage_name == "__raw__":
        return _ensure_df(X)
    tp = _truncate_pipeline(pipeline, stage_name)
    if tp is None:
        return _ensure_df(X)
    try:
        tp.set_output(transform="pandas")
    except Exception:
        pass
    Xt = tp.fit_transform(X)
    return _ensure_df(Xt)

def _ensure_df(X) -> pd.DataFrame:
    if isinstance(X, pd.DataFrame):
        return X
    if hasattr(X, "toarray"):
        X = X.toarray()
    if isinstance(X, np.ndarray):
        return pd.DataFrame(X, columns=[f"f{i}" for i in range(X.shape[1])])
    return pd.DataFrame(X)

def _expand_attributes_in_transformed(X_after: pd.DataFrame, attributes: Sequence[str]) -> List[str]:
    """Map requested raw attributes to transformed columns (handles one-hot by substring match)."""
    cols = []
    for a in attributes:
        a_low = str(a).lower()
        for c in X_after.columns:
            if a_low in str(c).lower():
                cols.append(c)
    if not cols:
        cols = [c for c in X_after.columns if c in attributes]
    # dedupe
    seen, out = set(), []
    for c in cols:
        if c not in seen:
            out.append(c); seen.add(c)
    return out


# ===========================================
# 1) Correlations before vs after any stage
# ===========================================
def compare_attribute_correlations(
    pipeline,
    X: pd.DataFrame,
    attributes: Sequence[str],
    stage_name: str,
    method: str = "pearson",
    figsize: Tuple[int, int] = (10, 4)
) -> Dict[str, pd.DataFrame]:
    """
    Compute & display correlation matrices for selected `attributes`
    BEFORE vs AFTER a given preprocessing `stage_name`.
    stage_name: "__raw__" | any pipeline step name | "__end__"
    """
    X_before = _ensure_df(X)
    X_after  = _fit_transform_at_stage(pipeline, X, stage_name)

    before_cols = [c for c in X_before.columns if c in attributes]
    after_cols  = _expand_attributes_in_transformed(X_after, attributes)

    corr_before = _ensure_df(X_before[before_cols]).corr(method=method) if before_cols else pd.DataFrame()
    corr_after  = _ensure_df(X_after[after_cols]).corr(method=method)   if after_cols  else pd.DataFrame()

    if not corr_before.empty:
        plt.figure(figsize=figsize)
        plt.imshow(corr_before, interpolation="nearest")
        plt.xticks(range(len(corr_before.columns)), corr_before.columns, rotation=90)
        plt.yticks(range(len(corr_before.index)), corr_before.index)
        plt.title(f"Correlation BEFORE stage '{stage_name}' ({method})")
        plt.colorbar()
        plt.tight_layout()
        plt.show()

    if not corr_after.empty:
        plt.figure(figsize=figsize)
        plt.imshow(corr_after, interpolation="nearest")
        plt.xticks(range(len(corr_after.columns)), corr_after.columns, rotation=90)
        plt.yticks(range(len(corr_after.index)), corr_after.index)
        plt.title(f"Correlation AFTER stage '{stage_name}' ({method})")
        plt.colorbar()
        plt.tight_layout()
        plt.show()

    return {"before": corr_before, "after": corr_after}


# ==================================================
# 2) Feature distribution before vs after any stage
# ==================================================
def compare_feature_distribution(
    pipeline,
    X: pd.DataFrame,
    feature: str,
    stage_name: str,
    bins: int = 20,
    top_k_categories: int = 25,
    figsize: Tuple[int, int] = (8, 4)
) -> Dict[str, pd.Series]:
    """
    Display the distribution of a single `feature` BEFORE vs AFTER `stage_name`.
    Numeric => histogram; Categorical => top-k bar counts.
    If transformed into multiple one-hot columns, aggregate those columns into a categorical count vector.
    """
    X_before = _ensure_df(X).copy()
    X_after  = _fit_transform_at_stage(pipeline, X, stage_name)

    # BEFORE
    if feature in X_before.columns:
        series_before = X_before[feature]
    else:
        matches = [c for c in X_before.columns if c.lower() == feature.lower()]
        series_before = X_before[matches[0]] if matches else pd.Series(dtype=float, name=feature)

    # AFTER (map to columns)
    after_cols = _expand_attributes_in_transformed(X_after, [feature])

    if len(after_cols) == 0:
        series_after = pd.Series(dtype=float, name=feature)
    elif len(after_cols) == 1 and after_cols[0] in X_after.columns:
        series_after = X_after[after_cols[0]]
    else:
        sub = X_after[after_cols]
        cat_names = [c.replace(f"{feature}_", "") for c in sub.columns]
        counts = sub.apply(pd.Series.sum, axis=0)
        counts.index = cat_names
        series_after = counts

    # Plot BEFORE
    if pd.api.types.is_numeric_dtype(series_before):
        plt.figure(figsize=figsize)
        plt.hist(series_before.dropna(), bins=bins)
        plt.title(f"Distribution BEFORE stage '{stage_name}' — {feature}")
        plt.xlabel(feature); plt.ylabel("count")
        plt.tight_layout(); plt.show()
        dist_before = series_before
    else:
        vc = series_before.astype("object").fillna("<NA>").value_counts().sort_values(ascending=False)
        vc = vc.head(top_k_categories)
        plt.figure(figsize=figsize)
        plt.bar(vc.index.astype(str), vc.values)
        plt.title(f"Distribution BEFORE stage '{stage_name}' — {feature}")
        plt.xticks(rotation=90); plt.ylabel("count")
        plt.tight_layout(); plt.show()
        dist_before = vc

    # Plot AFTER
    if isinstance(series_after, pd.Series) and pd.api.types.is_numeric_dtype(series_after) and series_after.shape[0] == X_after.shape[0]:
        plt.figure(figsize=figsize)
        plt.hist(series_after.dropna(), bins=bins)
        plt.title(f"Distribution AFTER stage '{stage_name}' — {feature}")
        plt.xlabel(feature); plt.ylabel("count")
        plt.tight_layout(); plt.show()
        dist_after = series_after
    else:
        if isinstance(series_after, pd.Series) and series_after.index.size > 0:
            counts = series_after.sort_values(ascending=False).head(top_k_categories)
            plt.figure(figsize=figsize)
            plt.bar(counts.index.astype(str), counts.values)
            plt.title(f"Distribution AFTER stage '{stage_name}' — {feature} (aggregated)")
            plt.xticks(rotation=90); plt.ylabel("count")
            plt.tight_layout(); plt.show()
            dist_after = counts
        else:
            dist_after = pd.Series(dtype=float, name=feature)

    return {"before": dist_before, "after": dist_after}


# ==========================================================
# 3) Threshold guard that calls YOUR mitigation function
#    - Uses your Fairness_Metrics_Computation(...)
#    - Calls your run_mitigations_and_predict(...) if violated
# ==========================================================
def _check_thresholds(metrics: Dict[str, float], thresholds: Dict[str, Tuple]) -> bool:
    """
    Return True if ANY threshold is violated.
    thresholds[k] = (op, val) or ("between", lo, hi)
    """
    def violated_one(metric, rule):
        op = rule[0]
        vals = rule[1:]
        m = metrics.get(metric, None)
        if m is None:
            return False
        if op == ">=":   return not (m >= vals[0])
        if op == "<=":   return not (m <= vals[0])
        if op == ">":    return not (m >  vals[0])
        if op == "<":    return not (m <  vals[0])
        if op == "between":
            lo, hi = vals
            return not (lo <= m <= hi)
        raise ValueError(f"Unknown threshold op: {op}")
    return any(violated_one(k, v) for k, v in thresholds.items())

def guard_and_mitigate_if_needed(
    pipeline,
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_test: pd.DataFrame,
    y_test: np.ndarray,
    sensitive_attr_test: np.ndarray,
    thresholds: Dict[str, Tuple],
    # ex:
    # thresholds = {
    #   "accuracy": (">=", 0.85),
    #   "disparate_impact_ratio": (">=", 0.8),
    #   "demographic_parity_diff": ("between", -0.1, 0.1)
    # }
    mitigations_cfg: List[Dict[str, Any]],
    context: Dict[str, Any],
    base_fit_params: Optional[Dict[str, Any]] = None,
    fairness_compute_type: str = "global",
    y2_scores: Optional[np.ndarray] = None,
    positive_label=1
) -> Dict[str, Any]:
    """
    1) Fit the baseline pipeline and compute metrics
       - Accuracy via sklearn
       - Fairness via your Fairness_Metrics_Computation(y1, y2, y, attribute, compute_type)
         (y1 = y_pred, y2 = y2_scores or None, y = y_test, attribute = sensitive_attr_test)
    2) If ANY threshold is violated, call your run_mitigations_and_predict(...)
    3) Return both pre/post metrics and final predictions
    """
    base_fit_params = dict(base_fit_params or {})
    model = clone(pipeline)
    model.fit(X_train, y_train, **base_fit_params)
    y_pred = model.predict(X_test)

    # --- metrics (accuracy + fairness) ---
    metrics = {"accuracy": float(accuracy_score(y_test, y_pred))}
    fairness_dict = Fairness_Metrics_Computation(
        y1=y_pred, y2=y2_scores, y=y_test, attribute=sensitive_attr_test, compute_type=fairness_compute_type
    )
    # Merge fairness metrics into metrics dict
    metrics.update({k: float(v) for k, v in fairness_dict.items()})

    violated = _check_thresholds(metrics, thresholds)

    result = {
        "violated": violated,
        "metrics_pre": metrics,
        "y_pred_pre": y_pred,
        "pipeline_pre": model
    }

    if not violated:
        result.update({
            "used_post_mitigation": False,
            "y_pred_final": y_pred,
            "metrics_final": metrics,
            "pipeline_final": model
        })
        return result

    # --- run your mitigation routine (already defined in your codebase) ---
    mit_res = run_mitigations_and_predict(
        pipeline=model,
        X_train=X_train, y_train=y_train,
        X_test=X_test,  y_test=y_test,
        mitigations=mitigations_cfg,
        base_fit_params=base_fit_params,
        context=context
    )

    # recompute fairness on post-mitigation predictions for apples-to-apples
    y_pred_post = mit_res["y_pred_post"]
    metrics_post = {"accuracy": float(accuracy_score(y_test, y_pred_post))}
    fairness_post = Fairness_Metrics_Computation(
        y1=y_pred_post, y2=y2_scores, y=y_test, attribute=sensitive_attr_test, compute_type=fairness_compute_type
    )
    metrics_post.update({k: float(v) for k, v in fairness_post.items()})

    result.update({
        "used_post_mitigation": True,
        "mitigation_detail": mit_res.get("applied_mitigations", []),
        "y_pred_post": y_pred_post,
        "metrics_post": metrics_post,
        "y_pred_final": y_pred_post,
        "metrics_final": metrics_post,
        "pipeline_final": mit_res["pipeline_final"],
    })
    return result


In [ ]:
compare_attribute_correlations(
    pipeline=pipeline_1,
    X=X_train,
    attributes=["sex"],
    stage_name="preprocess"    # or "__raw__", "__end__", or any step name in your pipeline
)


compare_feature_distribution(
    pipeline=pipeline_2,
    X=X_train,
    feature="education",
    stage_name="preprocess"
)

thresholds = {
    "accuracy": (">=", 0.85),
    "disparate_impact_ratio": (">=", 0.8),
    "demographic_parity_diff": ("between", -0.1, 0.1),
}


In [ ]:
A1n= accuracy_score(y_test, y_pred_1n)
A2n= accuracy_score(y_test, y_pred_2n)
f1n = f1_score(y_test, y_pred_1n)
f2n=f1_score(y_test, y_pred_2n)
A1, A1n



In [ ]:
from sklearn.dummy import DummyClassifier
import matplotlib.pyplot as plt

def calculate_biases(mispred_p1, mispred_p2, mispred_baseline):
    """
    Calculate individual biases and total bias.
    """
    # Calculating individual biases
    BD = mispred_baseline
    BStage = mispred_p1 - mispred_p2  # Preprocessing stage bias
    BC = mispred_p2 - mispred_baseline  # Classifier bias

    # Calculating Stage-Classifier Interaction term
    Stage_Classifier_Interaction = mispred_p1 - (BD + BStage + BC)

    # Total bias calculation
    TB = BD + BStage + BC + Stage_Classifier_Interaction

    return {
        'BD': BD,
        'BC': BC,
        'BStage': BStage,
        'Stage_Classifier_Interaction': Stage_Classifier_Interaction,
        'TB': TB,
        'TB matches mispred_p1': TB == mispred_p1
    }

def plot_biases(biases, mispred_p1):
    """
    Plot the biases and observed mispredictions.
    """
    labels = [
        'Data (BD)', 
         
        'Preprocessing Stage (BStage)', 
        'Stage-Classifier Interaction', 
        'Classifier (BC)',
        'Total Bias (TB)', 
        'Observed Bias (mispred_p1)'
    ]
    
    values = [
        biases['BD'], 
         
        biases['BStage'], 
        biases['Stage_Classifier_Interaction'], 
        biases['BC'],
        biases['TB'], 
        mispred_p1
    ]

    colors = ['blue', 'orange', 'green', 'red', 'pink', 'gray']

    plt.figure(figsize=(12, 8))
    bars = plt.barh(labels, values, color=colors)
    plt.xlabel('Bias / Mispredictions')
    plt.title('Bias Analysis and Observed Mispredictions - After Mitigation  ')

    # Adding annotations
    for bar in bars:
        width = bar.get_width()
        label_x_pos = width if width < 0 else width
        plt.text(
            label_x_pos, bar.get_y() + bar.get_height() / 2,
            f'{width:.2f}', ha='center', va='center',
            bbox=dict(facecolor='white', alpha=0.7)
        )

    plt.axvline(0, color='black', linewidth=0.8)
    plt.xlim(min(values) - 10, max(values) + 10)  # Adding some padding to the x-axis
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    plt.show()

# Baseline model for comparison


# Calculate mispredictions
aligned_indices = X_test.notna().all(axis=1)

mispred_p1n = sum(y_test != y_pred_1n)
mispred_p2n = sum(y_test != y_pred_2n)
# mispred_baseline = sum(y_test != y_pred_baseline)

# Calculate biases
biases = calculate_biases(mispred_p1n, mispred_p2n, mispred_baseline)

# Plot the biases
plot_biases(biases, mispred_p1n)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming y_pred_1, y_pred_2, y_pred_baseline, and y_test are already defined
# Convert y_test and predictions to numpy arrays to avoid indexing issues
y_test = np.array(y_test)
y_pred_1 = np.array(y_pred_1n)
y_pred_2 = np.array(y_pred_2n)
# y_pred_baseline = np.array(y_pred_baseline)

# Initialize volatility arrays
volatility_LE = np.zeros(len(y_test))
volatility_classifier = np.zeros(len(y_test))

# Calculate Volatility Scores for LE (Vol_St1)
for i in range(len(y_test)):
    if y_pred_1[i] == y_test[i] and y_pred_2[i] == y_test[i]:
        volatility_LE[i] = 0
    elif y_pred_1[i] == y_test[i] and y_pred_2[i] != y_test[i]:
        volatility_LE[i] = 1
    elif y_pred_1[i] != y_test[i] and y_pred_2[i] == y_test[i]:
        volatility_LE[i] = 2
    elif y_pred_1[i] != y_test[i] and y_pred_2[i] != y_test[i] and y_pred_1[i] == y_pred_2[i]:
        volatility_LE[i] = 3
    elif y_pred_1[i] != y_test[i] and y_pred_2[i] != y_test[i] and y_pred_1[i] != y_pred_2[i]:
        volatility_LE[i] = 4

# Volatility score calculation for Classifier (Vol_Cl)
for i in range(len(y_test)):
    if y_pred_1[i] == y_test[i] and y_pred_baseline[i] == y_test[i]:
        volatility_classifier[i] = 0
    elif y_pred_1[i] == y_test[i] and y_pred_baseline[i] != y_test[i]:
        volatility_classifier[i] = 1
    elif y_pred_1[i] != y_test[i] and y_pred_baseline[i] == y_test[i]:
        volatility_classifier[i] = 2
    elif y_pred_1[i] != y_test[i] and y_pred_baseline[i] != y_test[i] and y_pred_1[i] == y_pred_baseline[i]:
        volatility_classifier[i] = 3
    elif y_pred_1[i] != y_test[i] and y_pred_baseline[i] != y_test[i] and y_pred_1[i] != y_pred_baseline[i]:
        volatility_classifier[i] = 4

# Summarize volatility scores
volatility_summary = {
    'Condition': ['Vol_St1=0', 'Vol_St1=1', 'Vol_St1=2', 'Vol_St1=3', 'Vol_St1=4',
                  'Vol_Cl=0', 'Vol_Cl=1', 'Vol_Cl=2', 'Vol_Cl=3', 'Vol_Cl=4'],
    'Count': [np.sum(volatility_LE == 0), np.sum(volatility_LE == 1), np.sum(volatility_LE == 2), np.sum(volatility_LE == 3), np.sum(volatility_LE == 4),
              np.sum(volatility_classifier == 0), np.sum(volatility_classifier == 1), np.sum(volatility_classifier == 2), np.sum(volatility_classifier == 3), np.sum(volatility_classifier == 4)]
}

df_volatility = pd.DataFrame(volatility_summary)

# Plot the volatility scores
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Condition', y='Count', data=df_volatility, palette='viridis')

plt.title('Volatility Score Analysis  After Mitigation')
plt.xlabel('Condition')
plt.ylabel('Count')

# Add labels to each bar
for bar in ax.patches:
    ax.text(bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            f'{int(bar.get_height())}',
            ha='center', va='bottom', fontsize=11)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# Function to calculate accuracy and F1 score
def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return accuracy, f1

# Example computed metrics for illustration
accuracy_1= A1
accuracy_2 = np.abs(A1- A2)

accuracy_1n = A1n 
accuracy_2n = np.abs(A1n-A2n)  # After mitigation accuracies
f1_1 = f1
f1_2 = np.abs(f1-f2)
f1_1n= f1n
f1_2n =np.abs(f1n-f2n)
# Function to plot the accuracy and F1 scores for pipelines and components before and after mitigation
def plot_metrics():
    labels = ['Pipeline', 'RW Component']  # Update this if needed for component names
    
    # Accuracy plot
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    before_mitigation_accuracy = [accuracy_1, accuracy_2]
    after_mitigation_accuracy = [accuracy_1n, accuracy_2n]
    
    bar_width = 0.35  # Width of the bars
    x = np.arange(len(labels))
    
    plt.bar(x - bar_width / 2, before_mitigation_accuracy, width=bar_width, label='Before Mitigation', color='blue')
    plt.bar(x + bar_width / 2, after_mitigation_accuracy, width=bar_width, label='After Mitigation', color='green')
    
    plt.xticks(x, labels)
    plt.ylabel('Accuracy')
    plt.title('Accuracy Before vs After Mitigation')
    plt.legend()

    # F1 Score plot
    plt.subplot(1, 2, 2)
    before_mitigation_f1 = [f1_1, f1_2]
    after_mitigation_f1 = [f1_1n, f1_2n]
    
    plt.bar(x - bar_width / 2, before_mitigation_f1, width=bar_width, label='Before Mitigation', color='blue')
    plt.bar(x + bar_width / 2, after_mitigation_f1, width=bar_width, label='After Mitigation', color='green')
    
    plt.xticks(x, labels)
    plt.ylabel('F1 Score')
    plt.title('F1 Score Before vs After Mitigation')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage
plot_metrics()


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier
import numpy as np

# Load the data
file_path = 'C:/Users/Saadia/FairPreprocessing/data/bank/bank-additional-full.csv'
na_values = ['unknown', 'nan', '']  # More clearly defined missing value indicators
df = pd.read_csv(file_path, sep=';', na_values=na_values)
na_values = ['unknown']
df = pd.read_csv(file_path, sep=';', na_values=na_values)
df['age_mapped'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)
# Apply custom transformations
df['age'] = df['age'].apply(lambda x: float(x >= 25))  # Transforming 'age' into a binary feature
X = df.drop(columns=['y'])
y = df['y']

# Convert 'y' labels to numeric
y = y.map({'no': 0, 'yes': 1})

# Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Identify categorical and numerical columns
# categorical_cols = X_train.select_dtypes(include=['object']).columns
# numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

# Define a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ],
    remainder='passthrough'
)

# Manually compute sample weights for handling class imbalance
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# Define a pipeline incorporating Random Forest with class weight balancing
pipeline_1 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=150, class_weight='balanced'))
])

# Define a second pipeline without class weight balancing
pipeline_2 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))
])

def compute_propensity_scores(X, treatment_column):
    imputer = SimpleImputer(strategy='most_frequent')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    # Define the treatment variable
    treatment = X_imputed[treatment_column].apply(lambda x: 1 if x == 1 else 0)

    # Check if treatment has more than one class
    if treatment.nunique() < 2:
        raise ValueError(f"The treatment column '{treatment_column}' contains only one class: {treatment.unique()}. Propensity score calculation requires at least two classes.")
    
    # Drop the treatment column to create the covariate matrix
    X_covariates = X_imputed.drop(columns=[treatment_column])

    # One-hot encode the covariate matrix
    X_encoded = pd.get_dummies(X_covariates, drop_first=True)

    # Estimate propensity scores using logistic regression
    lr = LogisticRegression()
    lr.fit(X_encoded, treatment)
    propensity_scores = lr.predict_proba(X_encoded)[:, 1]

    return propensity_scores, treatment

def perform_matching(propensity_scores, treatment):
    # Define treated and control indices based on the treatment variable
    treated_indices = np.where((treatment == 1))[0] 
    control_indices = np.where((treatment == 0))[0]  

    # Fit the nearest neighbors model on control propensity scores
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(propensity_scores[control_indices].reshape(-1, 1))

    # Find the nearest neighbors for treated units
    distances, indices = nn.kneighbors(propensity_scores[treated_indices].reshape(-1, 1))

    # Map control indices to matched treated indices
    matched_control_indices = control_indices[indices.flatten()]

    # Combine treated and matched control indices
    matched_indices = np.concatenate([treated_indices, matched_control_indices])

    # Ensure indices are unique and within bounds
    matched_indices = np.unique(matched_indices)
    matched_indices = matched_indices[matched_indices < len(treatment)]

    return matched_indices


# Paths to train and test datasets
# Initialize lists to store results
spd_list = []
eod_list = []
aod_list = []
erd_list = []

spd_cas_list = []
eod_cas_list = []
aod_cas_list = []
erd_cas_list = []

spd_cf_list = []
eod_cf_list = []
aod_cf_list = []
erd_cf_list = []

for i in range(10):
    print(f"Iteration {i+1}")

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
    X_train_m = X_train.copy()
    X_test_m = X_test.copy()
    y_train_m = y_train.copy()
    y_test_m = y_test.copy()

    # Fit and predict using pipelines
    pipeline_1.fit(X_train, y_train)
    y_pred_1 = pipeline_1.predict(X_test)

    pipeline_2.fit(X_train, y_train)
    y_pred_2 = pipeline_2.predict(X_test)

    # Fairness Metrics Computation Placeholder
    SPD_mv, EOD_mv, AOD_mv, ERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['age_mapped'])

    spd_list.append(SPD_mv)
    eod_list.append(EOD_mv)
    aod_list.append(AOD_mv)
    erd_list.append(ERD_mv)

    # Counterfactual analysis: flip the age_mapped feature
    X_test_flipped = X_test.copy()
    X_test_flipped['age_mapped'] = flip_race(X_test['age_mapped'])

    y_pred_cf_1 = pipeline_1.predict(X_test_flipped)
    y_pred_cf_2 = pipeline_2.predict(X_test_flipped)

    # Compute counterfactual fairness metrics
    SPD_cf_mv, EOD_cf_mv, AOD_cf_mv, ERD_cf_mv = Fairness_Metrics_Computation(y_pred_cf_1, y_pred_cf_2, y_test, X_test_flipped['age_mapped'])

    spd_cf_list.append(SPD_cf_mv)
    eod_cf_list.append(EOD_cf_mv)
    aod_cf_list.append(AOD_cf_mv)
    erd_cf_list.append(ERD_cf_mv)

    # Propensity score computation and matching
    try:
        propensity_scores1, treatment1 = compute_propensity_scores(X_test_m, 'age_mapped')
        matched_indices1 = perform_matching(propensity_scores1, treatment1)
    except ValueError as e:
        print(e)
        continue  # Skip this iteration if the treatment column contains only one class

    # Propensity score computation and matching for Pipeline 2
    try:
        propensity_scores2, treatment2 = compute_propensity_scores(X_test_m, 'age_mapped')
        matched_indices2 = perform_matching(propensity_scores2, treatment2)
    except ValueError as e:
        print(e)
        continue  # Skip this iteration if the treatment column contains only one class

    # Find common matched indices across all pipelines
    common_matched_indices = set(matched_indices1).intersection(matched_indices2)
    common_matched_indices = list(common_matched_indices)

    # Ensure the indices are sorted to maintain order
    common_matched_indices.sort()

    # Applying matching to y_pred results and the actual y_test and age_mapped
    y1_matched = y_pred_1[common_matched_indices]
    y2_matched = y_pred_2[common_matched_indices]
    y_test_matched = y_test_m.reset_index(drop=True)[common_matched_indices]
    age_mapped_matched = X_test['age_mapped'].reset_index(drop=True)[common_matched_indices]

    SPD_cas_mv, EOD_cas_mv, AOD_cas_mv, ERD_cas_mv = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, age_mapped_matched)

    spd_cas_list.append(SPD_cas_mv)
    eod_cas_list.append(EOD_cas_mv)
    aod_cas_list.append(AOD_cas_mv)
    erd_cas_list.append(ERD_cas_mv)


In [ ]:
spd_array = np.array(spd_list)
eod_array = np.array(eod_list)
aod_array = np.array(aod_list)
erd_array = np.array(erd_list)

spd_cas_array = np.array(spd_cas_list)
eod_cas_array = np.array(eod_cas_list)
aod_cas_array = np.array(aod_cas_list)
erd_cas_array = np.array(erd_cas_list)

spd_cf_array = np.array(spd_cf_list)
eod_cf_array = np.array(eod_cf_list)
aod_cf_array = np.array(aod_cf_list)
erd_cf_array = np.array(erd_cf_list)


spd_mean = spd_array.mean(axis=0)
eod_mean = eod_array.mean(axis=0)
aod_mean = aod_array.mean(axis=0)
erd_mean = erd_array.mean(axis=0)

spd_cas_mean = spd_cas_array.mean(axis=0)
eod_cas_mean = eod_cas_array.mean(axis=0)
aod_cas_mean = aod_cas_array.mean(axis=0)
erd_cas_mean = erd_cas_array.mean(axis=0)

spd_cf_mean = spd_cf_array.mean(axis=0)
eod_cf_mean = eod_cf_array.mean(axis=0)
aod_cf_mean = aod_cf_array.mean(axis=0)
erd_cf_mean = erd_cf_array.mean(axis=0)

# Calculate the standard errors for each metric and stage
spd_se = spd_array.std(axis=0) / np.sqrt(spd_array.shape[0])
eod_se = eod_array.std(axis=0) / np.sqrt(eod_array.shape[0])
aod_se = aod_array.std(axis=0) / np.sqrt(aod_array.shape[0])
erd_se = erd_array.std(axis=0) / np.sqrt(erd_array.shape[0])

spd_cas_se = spd_cas_array.std(axis=0) / np.sqrt(spd_cas_array.shape[0])
eod_cas_se = eod_cas_array.std(axis=0) / np.sqrt(eod_cas_array.shape[0])
aod_cas_se = aod_cas_array.std(axis=0) / np.sqrt(aod_cas_array.shape[0])
erd_cas_se = erd_cas_array.std(axis=0) / np.sqrt(erd_cas_array.shape[0])

spd_cf_se = spd_cf_array.std(axis=0) / np.sqrt(spd_cf_array.shape[0])
eod_cf_se = eod_cf_array.std(axis=0) / np.sqrt(eod_cf_array.shape[0])
aod_cf_se = aod_cf_array.std(axis=0) / np.sqrt(aod_cf_array.shape[0])
erd_cf_se = erd_cf_array.std(axis=0) / np.sqrt(erd_cf_array.shape[0])

# Plotting function
def plot_with_error_bars(ax, means, std_errors, title, metric_labels, colors, labels):
    bar_width = 0.25
    indices = np.arange(len(metric_labels))  # Fix this to use metric_labels length
    
    for i, (mean, std_error, color, label) in enumerate(zip(means, std_errors, colors, labels)):
        ax.bar(indices + i * bar_width, mean, yerr=std_error, capsize=5, width=bar_width, align='center', color=color, label=label)
    
    ax.set_xticks(indices + bar_width)
    ax.set_xticklabels(metric_labels)
    ax.set_xlabel('Fairness Metrics')
    ax.set_ylabel('Values')
    ax.set_title(title)
    ax.legend()

# Metric labels
metric_labels = ['SPD', 'EOD', 'AOD', 'ERD']

# Create subplots
fig, axs = plt.subplots(1, 1, figsize=(18, 6))

# Colors and labels for the bars
colors = ['blue', 'orange', 'green']
labels = ['Statistical', 'Causal', 'Counterfactual']

# Prepare means and standard errors for plotting
means_mv = [spd_mean, eod_mean, aod_mean, erd_mean]
std_errors_mv = [spd_se, eod_se, aod_se, erd_se]
means_cas_mv = [spd_cas_mean, eod_cas_mean, aod_cas_mean, erd_cas_mean]
std_errors_cas_mv = [spd_cas_se, eod_cas_se, aod_cas_se, erd_cas_se]
means_cf_mv = [spd_cf_mean, eod_cf_mean, aod_cf_mean, erd_cf_mean]
std_errors_cf_mv = [spd_cf_se, eod_cf_se, aod_cf_se, erd_cf_se]

# Plot for MV
plot_with_error_bars(axs, [means_mv, means_cas_mv, means_cf_mv], [std_errors_mv, std_errors_cas_mv, std_errors_cf_mv], 'RW Impact', metric_labels, colors, labels)

# Prepare means and standard errors for plotting FE


In [ ]:
def Fairness_Metrics_Computation(y1, y2, y, attribute, compute_type='global'):
    """
    Compute local or global fairness metrics: SPD, EOD, AOD, ERD
    y1: Predictions from original pipeline (Y(P))
    y2: Predictions from modified pipeline (Y(P*))
    y: Ground truth labels
    attribute: Sensitive attribute (race)
    compute_type: 'local' for local metrics or 'global' for global metrics
    """
    # Convert inputs to pandas Series for easier handling
    y1 = pd.Series(y1).reset_index(drop=True)
    y2 = pd.Series(y2).reset_index(drop=True)
    y = pd.Series(y).reset_index(drop=True)
    attribute = pd.Series(attribute).reset_index(drop=True)

    # Identify privileged and unprivileged groups
    privileged = (attribute == 'male') | (attribute == 1)
    unprivileged = ~privileged

    if compute_type == 'local':
        # Local calculation: Only consider instances where y1 != y2
        local_indices = (y1 != y2).reset_index(drop=True)
        y1 = y1[local_indices]
        y2 = y2[local_indices]
        y = y[local_indices]
        attribute = attribute[local_indices]
        privileged = privileged[local_indices]
        unprivileged = unprivileged[local_indices]

    # Counts of privileged and unprivileged groups
    count_privileged = np.sum(privileged)
    count_unprivileged = np.sum(unprivileged)

    # SPD Calculation
    SFC_SPD_P = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                        for y1_i, y2_i, attr in zip(y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_SPD_NP = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                         for y1_i, y2_i, attr in zip(y1, y2, attribute) if attr != 'male' and attr != 1])

    SFR_SPD_p = SFC_SPD_P / count_privileged if count_privileged > 0 else 0
    SFR_SPD_np = SFC_SPD_NP / count_unprivileged if count_unprivileged > 0 else 0
    SPD = SFR_SPD_np - SFR_SPD_p

    # EOD Calculation
    P_Y1 = np.sum((y == 1) & privileged)
    NP_Y1 = np.sum((y == 1) & unprivileged)

    SFC_EOD_P = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_EOD_NP = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                         for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])

    SFR_EOD_p = SFC_EOD_P / P_Y1 if P_Y1 > 0 else 0
    SFR_EOD_np = SFC_EOD_NP / NP_Y1 if NP_Y1 > 0 else 0
    EOD = SFR_EOD_np - SFR_EOD_p

    # AOD Calculation
    P_Y0 = np.sum((y == 0) & privileged)
    NP_Y0 = np.sum((y == 0) & unprivileged)

    SFC_TP_P = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_TP_NP = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])

    SFR_AOD_p = (SFC_TP_P / P_Y1 if P_Y1 > 0 else 0) + (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0)
    SFR_AOD_np = (SFC_TP_NP / NP_Y1 if NP_Y1 > 0 else 0) + (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0)
    AOD = (SFR_AOD_np - SFR_AOD_p) / 2

    # ERD Calculation
    SFC_FN_P = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_FN_NP = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'male' or attr == 1])
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'male' and attr != 1])

    SFR_ERR_p = (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0) + (SFC_FN_P / P_Y1 if P_Y1 > 0 else 0)
    SFR_ERR_np = (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0) + (SFC_FN_NP / NP_Y1 if NP_Y1 > 0 else 0)
    ERD = SFR_ERR_np - SFR_ERR_p

    return SPD, EOD, AOD, ERD

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier
import numpy as np

# Load the data
file_path = 'C:/Users/Saadia/FairPreprocessing/data/bank/bank-additional-full.csv'
na_values = ['unknown']
df = pd.read_csv(file_path, sep=';', na_values=na_values)
df['age_mapped'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

# Separate Features and Target
X = df.drop(['y'], axis=1)
y = df['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Ensure 'age_mapped' is always present
X['age_mapped'] = df['age_mapped']

# Function to flip the binary race/age attribute
def flip_race(attribute):
    return attribute.apply(lambda x: 0 if x == 1 else 1)

categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Custom LabelEncoder that handles unseen labels
class CustomLabelEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.encoders = {}
        for col in X.columns:
            le = LabelEncoder()
            le.fit(X[col].astype(str))  # Ensure that all data is treated as strings
            self.encoders[col] = le
        return self

    def transform(self, X):
        X_encoded = X.copy()
        for col, le in self.encoders.items():
            unseen_label_value = -1
            X_encoded[col] = X[col].map(lambda x: le.transform([x])[0] if x in le.classes_ else unseen_label_value)
        return X_encoded

# Defining the pipelines
# pipeline_1 = Pipeline(steps=[
#     ('label_encoding', CustomLabelEncoder()),
#     ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
# ])

# pipeline_2 = Pipeline(steps=[
#     ('preprocessor', ColumnTransformer(
#         transformers=[
#             ('num', 'passthrough', numerical_cols),
#             ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
#         ]
#     )),
#     ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
# ])

def compute_propensity_scores(X, treatment_column):
    imputer = SimpleImputer(strategy='most_frequent')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    # Define the treatment variable
    treatment = X_imputed[treatment_column].apply(lambda x: 1 if x == 1 else 0)

    # Check if treatment has more than one class
    if treatment.nunique() < 2:
        raise ValueError(f"The treatment column '{treatment_column}' contains only one class: {treatment.unique()}. Propensity score calculation requires at least two classes.")
    
    # Drop the treatment column to create the covariate matrix
    X_covariates = X_imputed.drop(columns=[treatment_column])

    # One-hot encode the covariate matrix
    X_encoded = pd.get_dummies(X_covariates, drop_first=True)

    # Estimate propensity scores using logistic regression
    lr = LogisticRegression()
    lr.fit(X_encoded, treatment)
    propensity_scores = lr.predict_proba(X_encoded)[:, 1]

    return propensity_scores, treatment

def perform_matching(propensity_scores, treatment):
    # Define treated and control indices based on the treatment variable
    treated_indices = np.where((treatment == 1))[0] 
    control_indices = np.where((treatment == 0))[0]  

    # Fit the nearest neighbors model on control propensity scores
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(propensity_scores[control_indices].reshape(-1, 1))

    # Find the nearest neighbors for treated units
    distances, indices = nn.kneighbors(propensity_scores[treated_indices].reshape(-1, 1))

    # Map control indices to matched treated indices
    matched_control_indices = control_indices[indices.flatten()]

    # Combine treated and matched control indices
    matched_indices = np.concatenate([treated_indices, matched_control_indices])

    # Ensure indices are unique and within bounds
    matched_indices = np.unique(matched_indices)
    matched_indices = matched_indices[matched_indices < len(treatment)]

    return matched_indices


# Paths to train and test datasets
# Initialize lists to store results
spd_list = []
eod_list = []
aod_list = []
erd_list = []

spd_cas_list = []
eod_cas_list = []
aod_cas_list = []
erd_cas_list = []

gspd_list = []
geod_list = []
gaod_list = []
gerd_list = []

gspd_cas_list = []
geod_cas_list = []
gaod_cas_list = []
gerd_cas_list = []

for i in range(10):
    print(f"Iteration {i+1}")

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=X['age_mapped'])
    X_train_m = X_train.copy()
    X_test_m = X_test.copy()
    y_train_m = y_train.copy()
    y_test_m = y_test.copy()

    # Fit and predict using pipelines
    pipeline_1.fit(X_train, y_train)
    y_pred_1 = pipeline_1.predict(X_test)

    pipeline_2.fit(X_train, y_train)
    y_pred_2 = pipeline_2.predict(X_test)

    # Fairness Metrics Computation Placeholder
    SPD_mv, EOD_mv, AOD_mv, ERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['age_mapped'])

    spd_list.append(SPD_mv)
    eod_list.append(EOD_mv)
    aod_list.append(AOD_mv)
    erd_list.append(ERD_mv)

    # Counterfactual analysis: flip the age_mapped feature
    SPD_mv, EOD_mv, AOD_mv, ERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['age_mapped'] , compute_type='local')
   
    spd_list.append((SPD_mv))
    eod_list.append((EOD_mv))
    aod_list.append((AOD_mv))
    erd_list.append((ERD_mv))

    gSPD_mv, gEOD_mv, gAOD_mv, gERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['age_mapped'] , compute_type='global')
   
    gspd_list.append((gSPD_mv))
    geod_list.append((gEOD_mv))
    gaod_list.append((gAOD_mv))
    gerd_list.append((gERD_mv))
    
    propensity_scores1, treatment1 = compute_propensity_scores(X_test, 'age_mapped')
    matched_indices1 = perform_matching(propensity_scores1, treatment1)

    # Propensity score computation and matching for Pipeline 2
    propensity_scores2, treatment2 = compute_propensity_scores(X_test, 'age_mapped')
    matched_indices2 = perform_matching(propensity_scores2, treatment2)

    # Propensity score computation and matching for Pipeline 3
  
    # Find common matched indices across all pipelines
    common_matched_indices = set(matched_indices1).intersection(matched_indices2)
    common_matched_indices = list(common_matched_indices)

    # Ensure the indices are sorted to maintain order
    common_matched_indices.sort()

    # Applying matching to y_pred results and the actual y_test and Sex
    y1_matched = y_pred_1[common_matched_indices]
    y2_matched = y_pred_2[common_matched_indices]

    y_test_matched = y_test.reset_index(drop=True)[common_matched_indices]
    sex_matched = X_test['age_mapped'].reset_index(drop=True)[common_matched_indices]

    SPD_cas_mv, EOD_cas_mv, AOD_cas_mv, ERD_cas_mv = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched,compute_type='local')

#     # Causal fairness metrics for Feature Engineering impact
    spd_cas_list.append((SPD_cas_mv))
    eod_cas_list.append((EOD_cas_mv))
    aod_cas_list.append((AOD_cas_mv))
    erd_cas_list.append((ERD_cas_mv))   

    gSPD_cas_mv, gEOD_cas_mv, gAOD_cas_mv, gERD_cas_mv = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched,compute_type='global')

#     # Causal fairness metrics for Feature Engineering impact
    gspd_cas_list.append((gSPD_cas_mv))
    geod_cas_list.append((gEOD_cas_mv))
    gaod_cas_list.append((gAOD_cas_mv))
    gerd_cas_list.append((gERD_cas_mv))   
    
 

In [ ]:
def compute_means_and_errors(array):
    mean = array.mean(axis=0) * 0.1
    se = (array.std(axis=0) / np.sqrt(array.shape[0])) * 0.1
    return np.round(mean, 3), np.round(se, 3)


spd_array = np.array(spd_list)
eod_array = np.array(eod_list)
aod_array = np.array(aod_list)
erd_array = np.array(erd_list)

gspd_array = np.array(gspd_list)
geod_array = np.array(geod_list)
gaod_array = np.array(gaod_list)
gerd_array = np.array(gerd_list)

spd_cas_array = np.array(spd_cas_list)
eod_cas_array = np.array(eod_cas_list)
aod_cas_array = np.array(aod_cas_list)
erd_cas_array = np.array(erd_cas_list)

gspd_cas_array = np.array(gspd_cas_list)
geod_cas_array = np.array(geod_cas_list)
gaod_cas_array = np.array(gaod_cas_list)
gerd_cas_array = np.array(gerd_cas_list)


spd_mean, spd_se = compute_means_and_errors(spd_array)
eod_mean, eod_se = compute_means_and_errors(eod_array)
aod_mean, aod_se = compute_means_and_errors(aod_array)
erd_mean, erd_se = compute_means_and_errors(erd_array)

gspd_mean, gspd_se = compute_means_and_errors(gspd_array)
geod_mean, geod_se = compute_means_and_errors(geod_array)
gaod_mean, gaod_se = compute_means_and_errors(gaod_array)
gerd_mean, gerd_se = compute_means_and_errors(gerd_array)

spd_cas_mean, spd_cas_se = compute_means_and_errors(spd_cas_array)
eod_cas_mean, eod_cas_se = compute_means_and_errors(eod_cas_array)
aod_cas_mean, aod_cas_se = compute_means_and_errors(aod_cas_array)
erd_cas_mean, erd_cas_se = compute_means_and_errors(erd_cas_array)

gspd_cas_mean, gspd_cas_se = compute_means_and_errors(gspd_cas_array)
geod_cas_mean, geod_cas_se = compute_means_and_errors(geod_cas_array)
gaod_cas_mean, gaod_cas_se = compute_means_and_errors(gaod_cas_array)
gerd_cas_mean, gerd_cas_se = compute_means_and_errors(gerd_cas_array)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming your fairness metrics arrays are correctly formatted
# Metric labels
metric_labels = ['SPD', 'EOD', 'AOD', 'ERD']

# Colors for the bars
colors = ['blue', 'orange']
labels = ['Local', 'Global']

# Plot function for fairness metrics
def plot_fairness_metrics(ax, local_means, global_means, local_se, global_se, title, metric_labels):
    x = np.arange(len(metric_labels))  # label locations
    width = 0.35  # the width of the bars

    rects1 = ax.bar(x - width/2, local_means, width, label='Local', yerr=local_se, capsize=5)
    rects2 = ax.bar(x + width/2, global_means, width, label='Global', yerr=global_se, capsize=5)

    # Add labels, title, and custom ticks
    ax.set_xlabel('Fairness Metrics')
    ax.set_ylabel('Values')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(metric_labels)
    ax.legend()

# Replace these with the actual means and errors for statistical and causal fairness
# Statistical Local and Global for SS
local_mean_stage_1 = [spd_mean, eod_mean, aod_mean, erd_mean]
local_se_stage_1 = [spd_se, eod_se, aod_se, erd_se]

global_mean_stage_1 = [gspd_mean, geod_mean, gaod_mean, gerd_mean]
global_se_stage_1 = [gspd_se, geod_se, gaod_se, gerd_se]

# Causal Local and Global for SS
causal_means_stage_1 = [spd_cas_mean, eod_cas_mean, aod_cas_mean, erd_cas_mean]
causal_se_stage_1 = [spd_cas_se ,eod_cas_se, aod_cas_se, erd_cas_se]

gcausal_means_stage_1 = [gspd_cas_mean, geod_cas_mean, gaod_cas_mean, gerd_cas_mean]
gcausal_se_stage_1 = [gspd_cas_se, geod_cas_se, gaod_cas_se, gerd_cas_se]



# Create subplots with 2 rows and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(12, 10))

# Plot 1: Statistical Local and Global Fairness for MV
plot_fairness_metrics(axs[1], local_mean_stage_1, global_mean_stage_1, local_se_stage_1, global_se_stage_1, 
                      'Causal Local and Global Fairness - RW', metric_labels)

# Plot 2: Causal Local and Global Fairness for MV
plot_fairness_metrics(axs[0], causal_means_stage_1, gcausal_means_stage_1, causal_se_stage_1, gcausal_se_stage_1, 
                      'Statistical Local and Global Fairness - RW', metric_labels)



# Adjust layout and show plot
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier
import numpy as np

# Load the data
file_path = 'C:/Users/Saadia/FairPreprocessing/data/bank/bank-additional-full.csv'
na_values = ['unknown']
df = pd.read_csv(file_path, sep=';', na_values=na_values)
df['age_mapped'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

# Separate Features and Target
X = df.drop(['y'], axis=1)
y = df['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Ensure 'age_mapped' is always present
X['age_mapped'] = df['age_mapped']

# Function to flip the binary race/age attribute
def flip_race(attribute):
    return attribute.apply(lambda x: 0 if x == 1 else 1)

categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Custom LabelEncoder that handles unseen labels
class CustomLabelEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.encoders = {}
        for col in X.columns:
            le = LabelEncoder()
            le.fit(X[col].astype(str))  # Ensure that all data is treated as strings
            self.encoders[col] = le
        return self

    def transform(self, X):
        X_encoded = X.copy()
        for col, le in self.encoders.items():
            unseen_label_value = -1
            X_encoded[col] = X[col].map(lambda x: le.transform([x])[0] if x in le.classes_ else unseen_label_value)
        return X_encoded

# Defining the pipelines
# pipeline_1 = Pipeline(steps=[
#     ('label_encoding', CustomLabelEncoder()),
#     ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
# ])

# pipeline_2 = Pipeline(steps=[
#     ('preprocessor', ColumnTransformer(
#         transformers=[
#             ('num', 'passthrough', numerical_cols),
#             ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
#         ]
#     )),
#     ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
# ])

def compute_propensity_scores(X, treatment_column):
    imputer = SimpleImputer(strategy='most_frequent')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    # Define the treatment variable
    treatment = X_imputed[treatment_column].apply(lambda x: 1 if x == 1 else 0)

    # Check if treatment has more than one class
    if treatment.nunique() < 2:
        raise ValueError(f"The treatment column '{treatment_column}' contains only one class: {treatment.unique()}. Propensity score calculation requires at least two classes.")
    
    # Drop the treatment column to create the covariate matrix
    X_covariates = X_imputed.drop(columns=[treatment_column])

    # One-hot encode the covariate matrix
    X_encoded = pd.get_dummies(X_covariates, drop_first=True)

    # Estimate propensity scores using logistic regression
    lr = LogisticRegression()
    lr.fit(X_encoded, treatment)
    propensity_scores = lr.predict_proba(X_encoded)[:, 1]

    return propensity_scores, treatment

def perform_matching(propensity_scores, treatment):
    # Define treated and control indices based on the treatment variable
    treated_indices = np.where((treatment == 1))[0] 
    control_indices = np.where((treatment == 0))[0]  

    # Fit the nearest neighbors model on control propensity scores
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(propensity_scores[control_indices].reshape(-1, 1))

    # Find the nearest neighbors for treated units
    distances, indices = nn.kneighbors(propensity_scores[treated_indices].reshape(-1, 1))

    # Map control indices to matched treated indices
    matched_control_indices = control_indices[indices.flatten()]

    # Combine treated and matched control indices
    matched_indices = np.concatenate([treated_indices, matched_control_indices])

    # Ensure indices are unique and within bounds
    matched_indices = np.unique(matched_indices)
    matched_indices = matched_indices[matched_indices < len(treatment)]

    return matched_indices


# Paths to train and test datasets
# Initialize lists to store results
spd_list = []
eod_list = []
aod_list = []
erd_list = []

spd_cas_list = []
eod_cas_list = []
aod_cas_list = []
erd_cas_list = []

gspd_list = []
geod_list = []
gaod_list = []
gerd_list = []

gspd_cas_list = []
geod_cas_list = []
gaod_cas_list = []
gerd_cas_list = []

for i in range(3):
    print(f"Iteration {i+1}")

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=X['age_mapped'])
    X_train_m = X_train.copy()
    X_test_m = X_test.copy()
    y_train_m = y_train.copy()
    y_test_m = y_test.copy()

    # Fit and predict using pipelines
    pipeline_1.fit(X_train, y_train)
    y_pred_1 = pipeline_1.predict(X_test)

    pipeline_2.fit(X_train, y_train)
    y_pred_2 = pipeline_2.predict(X_test)
    
    res1 = guard_and_mitigate_if_needed(
        pipeline=pipeline_1,
        X_train=X_train, y_train=y_train,
        X_test=X_test,   y_test=y_test,
        sensitive_attr_test=s_test,
        thresholds=thresholds,
        mitigations_cfg=mitigations_cfg,  # the same list you pass to run_mitigations_and_predict
        context=common_context,           # e.g., {"s_train": s_train, ...}
        fairness_compute_type="global",   # or "local"
        y2_scores=None                    # provide scores if your fairness function needs them
    )
    y_pred_1 = res1["y_pred_final"]

    res2 = guard_and_mitigate_if_needed(
        pipeline=pipeline_2, X_train=X_train, y_train=y_train,
        X_test=X_test, y_test=y_test, sensitive_attr_test=s_test,
        thresholds=thresholds, mitigations_cfg=mitigations_cfg, context=common_context
    )
    y_pred_2 = res2["y_pred_final"]


    # Fairness Metrics Computation Placeholder
    SPD_mv, EOD_mv, AOD_mv, ERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['age_mapped'])

    spd_list.append(SPD_mv)
    eod_list.append(EOD_mv)
    aod_list.append(AOD_mv)
    erd_list.append(ERD_mv)

    # Counterfactual analysis: flip the age_mapped feature
    SPD_mv, EOD_mv, AOD_mv, ERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['age_mapped'] , compute_type='local')
   
    spd_list.append((SPD_mv))
    eod_list.append((EOD_mv))
    aod_list.append((AOD_mv))
    erd_list.append((ERD_mv))

    gSPD_mv, gEOD_mv, gAOD_mv, gERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['age_mapped'] , compute_type='global')
   
    gspd_list.append((gSPD_mv))
    geod_list.append((gEOD_mv))
    gaod_list.append((gAOD_mv))
    gerd_list.append((gERD_mv))
    
    propensity_scores1, treatment1 = compute_propensity_scores(X_test, 'age_mapped')
    matched_indices1 = perform_matching(propensity_scores1, treatment1)

    # Propensity score computation and matching for Pipeline 2
    propensity_scores2, treatment2 = compute_propensity_scores(X_test, 'age_mapped')
    matched_indices2 = perform_matching(propensity_scores2, treatment2)

    # Propensity score computation and matching for Pipeline 3
  
    # Find common matched indices across all pipelines
    common_matched_indices = set(matched_indices1).intersection(matched_indices2)
    common_matched_indices = list(common_matched_indices)

    # Ensure the indices are sorted to maintain order
    common_matched_indices.sort()

    # Applying matching to y_pred results and the actual y_test and Sex
    y1_matched = y_pred_1[common_matched_indices]
    y2_matched = y_pred_2[common_matched_indices]

    y_test_matched = y_test.reset_index(drop=True)[common_matched_indices]
    sex_matched = X_test['age_mapped'].reset_index(drop=True)[common_matched_indices]

    SPD_cas_mv, EOD_cas_mv, AOD_cas_mv, ERD_cas_mv = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched,compute_type='local')

#     # Causal fairness metrics for Feature Engineering impact
    spd_cas_list.append((SPD_cas_mv))
    eod_cas_list.append((EOD_cas_mv))
    aod_cas_list.append((AOD_cas_mv))
    erd_cas_list.append((ERD_cas_mv))   

    gSPD_cas_mv, gEOD_cas_mv, gAOD_cas_mv, gERD_cas_mv = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched,compute_type='global')

#     # Causal fairness metrics for Feature Engineering impact
    gspd_cas_list.append((gSPD_cas_mv))
    geod_cas_list.append((gEOD_cas_mv))
    gaod_cas_list.append((gAOD_cas_mv))
    gerd_cas_list.append((gERD_cas_mv))   
    
 

In [ ]:
def compute_means_and_errors(array):
    mean = array.mean(axis=0) * 0.1
    se = (array.std(axis=0) / np.sqrt(array.shape[0])) * 0.1
    return np.round(mean, 3), np.round(se, 3)


spd_array = np.array(spd_list)
eod_array = np.array(eod_list)
aod_array = np.array(aod_list)
erd_array = np.array(erd_list)

gspd_array = np.array(gspd_list)
geod_array = np.array(geod_list)
gaod_array = np.array(gaod_list)
gerd_array = np.array(gerd_list)

spd_cas_array = np.array(spd_cas_list)
eod_cas_array = np.array(eod_cas_list)
aod_cas_array = np.array(aod_cas_list)
erd_cas_array = np.array(erd_cas_list)

gspd_cas_array = np.array(gspd_cas_list)
geod_cas_array = np.array(geod_cas_list)
gaod_cas_array = np.array(gaod_cas_list)
gerd_cas_array = np.array(gerd_cas_list)


spd_mean, spd_se = compute_means_and_errors(spd_array)
eod_mean, eod_se = compute_means_and_errors(eod_array)
aod_mean, aod_se = compute_means_and_errors(aod_array)
erd_mean, erd_se = compute_means_and_errors(erd_array)

gspd_mean, gspd_se = compute_means_and_errors(gspd_array)
geod_mean, geod_se = compute_means_and_errors(geod_array)
gaod_mean, gaod_se = compute_means_and_errors(gaod_array)
gerd_mean, gerd_se = compute_means_and_errors(gerd_array)

spd_cas_mean, spd_cas_se = compute_means_and_errors(spd_cas_array)
eod_cas_mean, eod_cas_se = compute_means_and_errors(eod_cas_array)
aod_cas_mean, aod_cas_se = compute_means_and_errors(aod_cas_array)
erd_cas_mean, erd_cas_se = compute_means_and_errors(erd_cas_array)

gspd_cas_mean, gspd_cas_se = compute_means_and_errors(gspd_cas_array)
geod_cas_mean, geod_cas_se = compute_means_and_errors(geod_cas_array)
gaod_cas_mean, gaod_cas_se = compute_means_and_errors(gaod_cas_array)
gerd_cas_mean, gerd_cas_se = compute_means_and_errors(gerd_cas_array)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming your fairness metrics arrays are correctly formatted
# Metric labels
metric_labels = ['SPD', 'EOD', 'AOD', 'ERD']

# Colors for the bars
colors = ['blue', 'orange']
labels = ['Local', 'Global']

# Plot function for fairness metrics
def plot_fairness_metrics(ax, local_means, global_means, local_se, global_se, title, metric_labels):
    x = np.arange(len(metric_labels))  # label locations
    width = 0.35  # the width of the bars

    rects1 = ax.bar(x - width/2, local_means, width, label='Local', yerr=local_se, capsize=5)
    rects2 = ax.bar(x + width/2, global_means, width, label='Global', yerr=global_se, capsize=5)

    # Add labels, title, and custom ticks
    ax.set_xlabel('Fairness Metrics')
    ax.set_ylabel('Values')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(metric_labels)
    ax.legend()

# Replace these with the actual means and errors for statistical and causal fairness
# Statistical Local and Global for SS
local_mean_stage_1 = [spd_mean, eod_mean, aod_mean, erd_mean]
local_se_stage_1 = [spd_se, eod_se, aod_se, erd_se]

global_mean_stage_1 = [gspd_mean, geod_mean, gaod_mean, gerd_mean]
global_se_stage_1 = [gspd_se, geod_se, gaod_se, gerd_se]

# Causal Local and Global for SS
causal_means_stage_1 = [spd_cas_mean, eod_cas_mean, aod_cas_mean, erd_cas_mean]
causal_se_stage_1 = [spd_cas_se ,eod_cas_se, aod_cas_se, erd_cas_se]

gcausal_means_stage_1 = [gspd_cas_mean, geod_cas_mean, gaod_cas_mean, gerd_cas_mean]
gcausal_se_stage_1 = [gspd_cas_se, geod_cas_se, gaod_cas_se, gerd_cas_se]



# Create subplots with 2 rows and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(12, 10))

# Plot 1: Statistical Local and Global Fairness for MV
plot_fairness_metrics(axs[1], local_mean_stage_1, global_mean_stage_1, local_se_stage_1, global_se_stage_1, 
                      'Causal Local and Global Fairness - RW', metric_labels)

# Plot 2: Causal Local and Global Fairness for MV
plot_fairness_metrics(axs[0], causal_means_stage_1, gcausal_means_stage_1, causal_se_stage_1, gcausal_se_stage_1, 
                      'Statistical Local and Global Fairness - RW', metric_labels)



# Adjust layout and show plot
plt.tight_layout()
plt.show()
